$$
\newcommand{\bolde}{\boldsymbol{e}}
\newcommand{\boldh}{\boldsymbol{h}}
\newcommand{\boldp}{\boldsymbol{p}}
\newcommand{\boldr}{\boldsymbol{r}}
\newcommand{\boldt}{\boldsymbol{t}}
\newcommand{\boldq}{\boldsymbol{q}}
\newcommand{\boldM}{\boldsymbol{M}}
\newcommand{\boldP}{\boldsymbol{P}}
\newcommand{\boldR}{\boldsymbol{R}}
\newcommand{\boldT}{\boldsymbol{T}}
\newcommand{\boldQ}{\boldsymbol{Q}}
$$

# High Definition (HD) Maps

<img src='./images/hd_maps_01.jpg' width='100%'>

<img src='./images/hd_maps_02.jpg' width='100%'>

## Настройка окружения

Для выполнения кода из данного ноутбука лучше создать отдельное питоновское окружение. Проблема в том, что далее надо будет поставить довольно много библиотек, и, к сожалению, возможна ситуация возникнования конфликтов между ними.

Существует два способа настроить питоновское окружение для запуска кода из данного ноутбука:
1. С помощью `conda`
2. С помощью `virtualenv`

Полную информацию по управлению окружениями `conda` можно найти на [странице документации](
https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html). Далее будет просто пошаговая инструкция конкретно для наших целей.

Мы воспользуемся первым, как более простым. Предполагается, что `conda` у вас уже установлена. Для создания нового окружения с именем `sdc` и версией питона 3.9 выполняем команду:
```shell
conda create -n sdc python=3.9
```

Если все прошло успешно, то `sdc` появится в списке окружений:
```shell
conda env list
```

Теперь переключаемся в консоли на окружение `sdc`:
```shell
conda activate sdc
```
В результате в командной строке будем видеть нечто такое:
```
(sdc) ~$
```

Теперь, когда мы успешно переключились на окружение `sdc`, начинаем устанавливаеть нужные библиотеки.

1. На всякий случай обновляем версии основных библиотек:
    ```shell
    pip install numpy --upgrade
    pip install scipy --upgrade
    pip install matplotlib --upgrade
    ```
2. Устанавливаем `jupyter lab` и `jupyter notebook` (`ipython-notebook`):
```shell
pip install jupyterlab
pip install notebook
```
3. Теперь самое &laquo;интересное&raquo; &mdash; ставим библиотеки, которые нужны конкретно для данного ноутбука:
    ```shell
    pip install opencv-python
    pip install plotly
    pip install open3d
    pip install pykitti
    pip install pyproj
    ```

Если все прошло успешно, то закрываем данный ноутбук, в консоли переключаемся на окружение `sdc`, и находясь в данном окружение запускаемся заново
```
(sdc) ...$ ls
hd_maps.ipynb ...
(sdc) ...$ jupyter notebook
```
После перезапуска все импорты ниже должны работать.

> Замечание. Если будут какие-то проблемы с 3D-визуализацией `plotly`, то стоит попробовать запустиь ноутбук с командой `--no-browser`
> ```shell
> (sdc) ...$ jupyter notebook --no-browser
> ```
> и уже затем открыть ссылку вида `http://127.0.0.1:8888/tree?token=...`

In [ ]:
# Специальные import-ы
import plotly
import open3d
import pykitti
import pyproj
import cv2

from IPython.display import (
    display,
    SVG,
    HTML,
    Image,
    Video,
    YouTubeVideo,
)

import typing as T
import os
import copy
import tqdm
import itertools
from collections import namedtuple
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#display(HTML("<style>.container { width:100% !important; }</style>"))

<a name='_toc'></a>
# Содержание
* [Лидары](#introduction)
    * [Примеры лидаров](#lidars_examples)
    * [Принцип работы](#lidars_operating_principles)
    * [Данные лидара](#lidars_data)
    * [Источники ошибок](#lidars_measurement_errors)
* [Лидарные облака](#lidars_clouds)
    * [Визуализация облаков](#lidars_clouds_visualization)
    * [Операции над облаками](#lidars_clouds_operations)
        * [Линейные преобразования](#linear_transformations)
        * [Фильтрация](#pcl_filtration)
        * [Оценка фичей](#pcl_features_estimation)
        * [Геометрическая сегментация](#geometrical_segmentation)
* [Оценка позы](#pose_estimation)
    * [Point set registration](#point_set_registration)
    * [Iterative Closest Point (ICP)](#icp)
* [KITTI](#kitti)
    * [Загрузка сцен](#kitti_loading)
    * [Содержимое сцен](#kitti_content)
        * [Изображения](#kitti_images)
        * [Облака](#kitti_clouds)
        * [Локализация](#kitti_localization)
        * [Калибровки](#kitti_calibrations)
* [Источники](#sources)

<a name='lidars'></a>
# Лидары<sup>[toc](#_toc)</sup>

> **LiDAR &mdash; Light Detection and Ranging**

* [Примеры лидаров](#lidars_manufacturers)

<a name='lidars_examples'></a>
## Примеры лидаров<sup>[toc](#_toc)</sup>
* [Velodyne](velodyne)
* [Waymo](waymo)
* [Yandex](yandex)

<a name='velodyne'></a>
### Velodyne<sup>[toc](#_toc)</sup>

In [ ]:
Image('./images/velodyne_lidar_00.png', width='100%')

In [ ]:
Image('./images/velodyne_lidar_01.png', width='100%')

In [ ]:
Image('./images/velodyne_lidar_cloud_HDL-64E.jpg', width='100%')

In [ ]:
YouTubeVideo('tZ8WbSNsNaU', width='100%', height=500)

<a name='waymo'></a>
### Waymo<sup>[toc](#_toc)</sup>

In [ ]:
Image('./images/waymo_lidar_cloud_00.png', width='100%')

<a name='yandex_lidars'></a>
### Yandex<sup>[toc](#_toc)</sup>

#### Модели<sup>[toc](#_toc)</sup>

In [ ]:
Image('./images/yandex_lidar_00.jpeg')

In [ ]:
Image('./images/yandex_lidar_01.jpeg')

In [ ]:
Image('./images/yandex_lidar_02.jpeg')

<a name='yandex_lidar_clouds'></a>
### Облака лидара<sup>[toc](#_toc)</sup>

**Так выглядит изображение с лидара. Можно увидеть дороги, пешеходов, припаркованные машины, стоянку самокатов и другие объекты вокруг беспилотного автомобиля**

In [ ]:
Image('./images/yandex_lidar_cloud_00.png', width='100%')

In [ ]:
Image('./images/yandex_lidar_cloud_01.jpeg', width='100%')

In [ ]:
Image('./images/yandex_lidar_cloud_02.jpeg', width='100%')

In [ ]:
Image('./videos/yandex_lidar_02.gif', width='100%')

In [ ]:
Video('./videos/Yandex Software-Defined LiDAR.mp4', width=970)

In [ ]:
YouTubeVideo('lPclzuDtmTs', width='100%', height=500)

<a name='lidars_operating_principles'></a>
## Принцип работы<sup>[toc](#_toc)</sup>
* [Time-of-flight ranging](#time_of_flight_ranging)
* [Интенсивность](#intensity)
* [Сканирование пространства](#scanning)

<a name='time_of_flight_ranging'></a>
### Time-of-flight ranging<sup>[toc](#_toc)</sup>

Лидар работает по принципу [**time-of-flight ranging**-а](https://en.wikipedia.org/wiki/Time-of-flight_camera). По такому же принципу работают радары и сонары.

Терминология:
* Time-of-Flight
* Time-of-Arrival
* Round trip time (RTT)
* Round trip distance (RTD)

In [ ]:
display(SVG('https://upload.wikimedia.org/wikipedia/commons/f/f1/20200501_Time_of_flight.svg'))

Пусть
* $t$ &mdash; время от отправки импулься до его получения, т.е. RTT (Round Trip Time)
* $c$ &mdash; скорость света
Тогда расстояние $r$ до мишени/объекта наблюдения можно __оценить__ следующим образом:
$$
r \approx \frac{c \Delta t}{2}
$$

<a name='intensity'></a>
### Интенсивность<sup>[toc](#_toc)</sup>

![](https://res.mdpi.com/sensors/sensors-15-28099/article_deploy/html/images/sensors-15-28099-g001-1024.png)

<a name='scanning'></a>
### Сканирование пространства<sup>[toc](#_toc)</sup>

Вращение зеркала-отражателя вокруг вертикальной оси позволяет получить 2d-срез окружающего пространства. Чтобы получить паттерн 3d-сканирования добавляют вращение зеркальца вокруг горизонтальной оси.
![](videos/general_rotating_lidar.gif)

Лучей может быть много:
![](https://eckop.com/wp-content/uploads/2018/07/LIDAR_02-1024x366.png)

Паттерн сканирования велодайна:

![](./images/velodyne_lidar_structure_00.png)

![](./images/velodyne_lidar_structure_01.png)

[Static Calibration and Analysis of the Velodyne HDL-64E S2 for High Accuracy Mobile Scanning](https://www.mdpi.com/2072-4292/2/6/1610)

<a name='lidars_data'></a>
## Данные с лидара<sup>[toc](#_toc)</sup>

### "Сырые" показания лидара<sup>[toc](#_toc)</sup>
* Горизонатльный поворот зеркальца (galvo horizontal encoder value)
* Вертикальный поворот зеркальца (galvo vertical encoder value)
* время полета (time-of-flight)

### Сферическая система координат<sup>[toc](#_toc)</sup>

Положение точки $P$ в сферической системе координат определяется тройкой $(r, \theta, \varphi)$, где
* $r \ge 0$ &mdash; **расстояние** (**range/distance**)
* $\theta \in [-\pi/2, \pi/2]$ &mdash; **зенитный** или **полярный** угол (**elevation angle**)
* $\varphi \in [0, 2\pi)$ &mdash; **азимутальный** угол (**azimuth angle**)

Иногда встречаются альтернативные обозначения:
* $r$ &mdash; **r**ange
* $\alpha$ &mdash; **a**zimuth angle
* $\varepsilon$ &mdash; **e**levation angle

In [ ]:
display(SVG('./images/spherical_coordinate_system.svg'))

### Декартова система координат<sup>[toc](#_toc)</sup>

\begin{align*}
\begin{pmatrix}
x\\
y\\
z
\end{pmatrix} =
\begin{pmatrix}
r \cos\theta \cos\phi \\ 
r \cos\theta \sin\phi\\
r \sin\theta
\end{pmatrix}
\end{align*}

\begin{align*}
\begin{pmatrix}
r\\
\phi\\
\theta\\
\end{pmatrix} = \boldh(x, y, z) = \begin{pmatrix}
\sqrt{x^2 + y^2 + z^2}\\
\arctan \left(\frac{y}{x}\right)\\
\arcsin \left(\frac{z}{\sqrt{x^2 + y^2 + z^2}}\right)
\end{pmatrix}
\end{align*}

<a name='lidars_measurement_errors'></a>
## Источники ошибок<sup>[toc](#_toc)</sup>
* Неточность в определении времени прибытия сигнала (из-за peak detector-а)
* Неточности в определении внутренних механических частей лидара (зеркальца и т.п.)
* Взаимодействие со средой
* Взаимодействие с поверхностью отражения

Пусть $\boldp = (x, y, z)^T  \in \mathbb{R}^3$ &mdash; реальная точка, от которой отразился сигнал, тогда в первом приближении можно моделировать ошибки наблюдения нормальным шумом:
$$
\begin{pmatrix}
\hat{r}\\
\hat{\phi}\\
\hat{\theta}\\
\end{pmatrix} = \hat{\boldh}(x, y, z) = \boldh(x, y, z) + \boldq,
$$
где $\boldq \sim \mathcal{N}(\boldsymbol{0}, Q)$.

Ещё одна причина искажений в наблюдениях &mdash; движение устройства, на котором установлен лидар (робота, машины). В таком случае могут возникать двоения, растяжени и сжатия объектов (впрочем, как и много всего прочего).

<a name='lidars_clouds'></a>
# Лидарные облака<sup>[toc](#_toc)</sup>
* [Визуализация облаков](#lidar_clouds_visualization)
* [Операции над облаками](#lidars_clouds_operations)

<a name='lidars_clouds_visualization'></a>
## Визуализация облаков<sup>[toc](#_toc)</sup>

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)

from pylib.pcl.tools.plotly_visualization import (
    create_plotly_figure,
    plot_cloud,
    apply_min_max_scaling,
    convert_values_to_rgba_tuples_f64,
)

#### Случайное облако<sup>[toc](#_toc)</sup>

In [ ]:
num_points = 100
cloud_xyz = np.random.normal(size=(num_points, 3))

colors = apply_min_max_scaling(cloud_xyz[:, 2])

colors = convert_values_to_rgba_tuples_f64(colors, cmap='viridis')
figure = create_plotly_figure(bgcolor='black')
plot_cloud(
    cloud_xyz,
    colors=colors,
    labels=np.linalg.norm(cloud_xyz, axis=1),
    marker_size=3, figure=figure).show()
del num_points, cloud_xyz, colors

#### Bunny<sup>[toc](#_toc)</sup>

In [ ]:
data = pd.read_csv('./datasets/pcl/bunny.txt', header=None, sep=' ', names=['x', 'y', 'z'])
cloud_xyz = data.to_numpy()
x = cloud_xyz[:, 0]
y = cloud_xyz[:, 2]
z = cloud_xyz[:, 1]
cloud_xyz = np.stack([x, y, z]).T
print(cloud_xyz.shape)
plot_cloud(cloud_xyz, marker_size=5).show()
del data, cloud_xyz

<a name='lidars_clouds_operations'></a>
## Операции над облаками<sup>[toc](#_toc)</sup>

In [ ]:
from scipy.spatial.transform import Rotation
from pylib.pcl.common.cloud_io import read_point_cloud_o3d, read_point_cloud_xyz
from pylib.pcl.common.convert_point_cloud import convert_point_cloud_o3d_to_xyz
from pylib.pcl.common.transform_point_cloud import transform_point_cloud_xyz
from pylib.pcl.filters.voxel_grid import apply_voxel_grid

<a name='linear_transformations'></a>
### Линейные преобразования<sup>[toc](#_toc)</sup>

**Базовые операции**:
* Сдвиг (translation)
* Поворот (rotation)
* Масштабирование (scaling)

**Композитные операции**:
* Афинное преобразование (используется редко, так как не физично)
* Изометрическое преобразование (__преобразование системы координат__)

<a name='example_stanford_bunny'></a>
#### Пример. Stanford Bunny<sup>[toc](#_toc)</sup>

http://graphics.stanford.edu/data/3Dscanrep/

> **Задача** Загрузить данные сканирования датасета и смержить в единое облако с использованием информации о том, из каких поз производилось сканирование

Читаем облака:

In [ ]:
dataset_path = './datasets/stanford/bunny/data'
clouds_names = [
    'bun000.ply',
    'bun045.ply',
    'bun090.ply',
    'bun180.ply',
    'bun270.ply',
    'top2.ply',
    'top3.ply',
    'bun315.ply',
    'chin.ply',
    'ear_back.ply',
]

cloud_o3d_by_name = {}
for cloud_name in clouds_names:
    cloud_o3d = read_point_cloud_o3d(os.path.join(dataset_path, cloud_name))
    cloud_o3d_by_name[cloud_name] = cloud_o3d
    print(cloud_o3d)
    del cloud_o3d

Читаем позы:

In [ ]:
conf_name = 'bun.conf'

pose_by_cloud_name = {}
with open(os.path.join(dataset_path, conf_name), 'r') as f:
    for line in f.readlines():
        line = line.split()
        if len(line) == 0:
            continue
        if line[0] != 'bmesh':
            continue
        _, cloud_name, x, y, z, qx, qy, qz, qw = line
        x = float(x)
        y = float(y)
        z = float(z)
        qx = float(qx)
        qy = float(qy)
        qz = float(qz)
        qw = float(qw)
        rotation_matrix = np.linalg.inv(Rotation.from_quat([qx, qy, qz, qw]).as_matrix())
        translation_vector = np.array([x, y, z])
        transform_matrix = np.eye(4)
        transform_matrix[:3, :3] = rotation_matrix
        transform_matrix[:3, 3] = translation_vector
        pose_by_cloud_name[cloud_name] = transform_matrix

Получаем XYZ-облака:

In [ ]:
cloud_xyz_by_name = {}
transformed_cloud_xyz_by_name = {}
for cloud_name, cloud_o3d in cloud_o3d_by_name.items():
    cloud_xyz_by_name[cloud_name] = convert_point_cloud_o3d_to_xyz(cloud_o3d)
    transformed_cloud_xyz_by_name[cloud_name] =\
        transform_point_cloud_xyz(cloud_xyz_by_name[cloud_name], pose_by_cloud_name[cloud_name])

Мержим трансформированные облака в единое облако:

In [ ]:
merged_cloud_xyz = []
for cloud_name in clouds_names:
    merged_cloud_xyz.append(transformed_cloud_xyz_by_name[cloud_name])
merged_cloud_xyz = np.vstack(merged_cloud_xyz)
print(f'Merged cloud shape: {merged_cloud_xyz.shape}')

Отрисовывать облако такого размера для `plotly` &mdash; довольно сложная задача. Поэтому уменьшим его размер с помощью вокселизации:

In [ ]:
print(f'Merged cloud shape before voxel grid: {merged_cloud_xyz.shape}')
merged_cloud_xyz = apply_voxel_grid(merged_cloud_xyz, voxel_size=0.0015)
print(f'Merged cloud shape after voxel grid: {merged_cloud_xyz.shape}')

Визуализируем смерженное вокселизируемое облако:

In [ ]:
figure = create_plotly_figure()
plot_cloud(merged_cloud_xyz, figure=figure)
figure.show()

<a name='pcl_filtration'></a>
### Фильтрация<sup>[toc](#_toc)<sup>
    
TODO

<a name='pcl_features_estimation'></a>
### Оценка признаков<sup>[toc](#_toc)</sup>

TODO

<a name='geometrical_segmentation'></a>
### Геометрическая сегментация<sup>[toc](#_toc)</sup>

TODO

<a name='pose_estimation'></a>
# Оценка позы<sup>[toc](#_toc)</sup>
* [Point set registration](#point_set_registration)
* [Iterative Closest Point (ICP)](#icp)
* [Проблемы ICP](#icp_problems)

## Point set registration<sup>[toc](#_toc)</sup>

Как заматчить два облака?

<a name='icp'></a>
## Iterative Closest Point<sup>[toc](#_toc)</sup>

https://www.open3d.org/docs/release/tutorial/pipelines/icp_registration.html

__Идея__. Если хороших соответствий больше, чем плохих, то в процессе оптмизации их будет становиться все больше вплоть до сходимости процесса

__Алгоритм__
1. Хорошее начальное приближение. Возможно из constant velocity model или zero velocity motion model
2. Построение соответствий (association, correspondences estimation)
3. Оценка трансформа (transform estimation)
4. Продолжаем итерации вплоть до сходимости

__Варианты ICP__
* Point to point
* Point to plane
* Plane to plane
* Point to distr
* Distr to distr (GICP)

<a name='icp_problems'></a>
## Проблемы ICP<sup>[toc](#_toc)</sup>
* Выбросы и шумы в самих облаках
* Неточные соответствия
* Движения машины
* Движение вокруг машины

<a name='kitti'></a>
# KITTI<sup>[toc](#_toc)</sup>

Датасеты:
* https://www.thinkautonomous.ai/blog/lidar-datasets/
* https://www.argoverse.org/av2.html
* https://www.kaggle.com/datasets/klemenko/kitti-dataset/data

Мы будем работать с KITTI. Основной сайт https://www.cvlibs.net/datasets/kitti/

## Конфигурация<sup>[toc](#_toc)</sup>

![](https://www.cvlibs.net/datasets/kitti/images/setup_top_view.png)

![](https://www.cvlibs.net/datasets/kitti/images/passat_sensors_920.png)

<a name='kitti_loading'></a>
## Загрузка сцен<sup>[toc](#_toc)</sup>
Нужно скачать набор данных со страницы http://www.cvlibs.net/datasets/kitti/raw_data.php

Содержимое датасетов:
* Raw (unsynced+unrectified) and processed (synced+rectified) grayscale stereo sequences (0.5 Megapixels, stored in png format)
* Raw (unsynced+unrectified) and processed (synced+rectified) color stereo sequences (0.5 Megapixels, stored in png format)
* 3D Velodyne point clouds (100k points per frame, stored as binary float matrix)
* 3D GPS/IMU data (location, speed, acceleration, meta information, stored as text file)
* Calibration (Camera, Camera-to-GPS/IMU, Camera-to-Velodyne, stored as text file)
* 3D object tracklet labels (cars, trucks, trams, pedestrians, cyclists, stored as xml file)

#### 2011_09_26_drive_0002 (0.3 GB)
* [unsynced+unrectified data](https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0002/2011_09_26_drive_0002_extract.zip)
* [synced+rectified data](https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0002/2011_09_26_drive_0002_sync.zip)
* [calibration](https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_calib.zip)
* [tracklets](https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0002/2011_09_26_drive_0002_tracklets.zip)

#### 2011_09_26_drive_0106 (0.9 GB)
* [unsynced+unrectified data](https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0106/2011_09_26_drive_0106_extract.zip)
* [synced+rectified data](https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_drive_0106/2011_09_26_drive_0106_sync.zip)
* [calibration](https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/2011_09_26_calib.zip)

Для удобной работы с выкаченным датасетом есть библиотека `pykitti` (https://github.com/utiasSTARS/pykitti)

In [ ]:
KITTI_DIR_PATH = os.path.abspath('./datasets/KITTI/')
print(f'KITTI dataset base dir: <{KITTI_DIR_PATH}>')

# Указываем датасет для загрузки
RIDE_DATE = '2011_09_26'
DRIVE = '0002'

# Загружаем данные. Опционально можно указать диапазон фреймов для загрузки
KITTI_DATASET = pykitti.raw(base_path=KITTI_DIR_PATH, date=RIDE_DATE, drive=DRIVE)

# Сразу же создаем адаптор
KITTI_DATASET_ADAPTOR = KittiDatasetAdaptor(KITTI_DATASET)

<a name='kitti_content'></a>
## Содержимое сцен<sup>[toc](#_toc)</sup>
* [Изображения](#kitti_images)
* [Облака](#kitti_clouds)
* [Локализация](#kitti_localization)
* [Калибровки](#kitti_calibrations)

Далее посмотрим на то, что содержится в загруженном датасете KITTI.

<a name='kitti_images'></a>
## Изображения<sup>[toc](#_toc)</sup>

**В датасете содержатся данные для 4-х камер**

In [ ]:
print(f'Number of camera 0 images: {len(KITTI_DATASET.cam0_files)}')
print(f'Number of camera 1 images: {len(KITTI_DATASET.cam1_files)}')
print(f'Number of camera 2 images: {len(KITTI_DATASET.cam2_files)}')
print(f'Number of camera 3 images: {len(KITTI_DATASET.cam3_files)}')

**Посмотрим на ректифицированные изображения камер:**

In [ ]:
frame_idx = 0
image0 = KITTI_DATASET.get_cam0(frame_idx)
image1 = KITTI_DATASET.get_cam1(frame_idx)
image2 = KITTI_DATASET.get_cam2(frame_idx)
image3 = KITTI_DATASET.get_cam3(frame_idx)

plt.figure(figsize=(20, 8))
plt.subplot(2, 2, 1)
plt.imshow(image0, cmap='gray')
plt.title('camera 0')

plt.subplot(2, 2, 2)
plt.imshow(image1, cmap='gray')
plt.title('camera 1')

plt.subplot(2, 2, 3)
plt.imshow(image2)
plt.title('camera 2')

plt.subplot(2, 2, 4)
plt.imshow(image3)
plt.title('camera 3');

plt.tight_layout()

del frame_idx, image0, image1, image2, image3

In [ ]:
frame_idx = 0
image = KITTI_DATASET.get_cam2(1)
print(f'Image type={type(image)}, image size={image.size}')
plt.figure(figsize=(16, 12))
plt.imshow(image)
del frame_idx, image

Ректифицированные изображения &mdash; это изображения в стерео-плоскости. Все камеры приведены к одной стерео-плоскости. Это нужно учитывать при проекции лидарных точек на ректифицированные изображения.

**Посмотрим на изображения цветной стерео-пары:**<sup>[toc](#_toc)</sup>

In [ ]:
frame_idx = 1
image1, image2 = KITTI_DATASET.get_rgb(frame_idx)
print(f'image1 type={type(image1)}, image1 size={image1.size}')
print(f'image2 type={type(image2)}, image2 size={image2.size}')

plt.figure(figsize=(16, 12))
plt.subplot(2, 1, 1)
plt.imshow(image1)
plt.subplot(2, 1, 2)
plt.imshow(image2);

<a name='kitti_clouds'></a>
### Облака<sup>[toc](#_toc)</sup>

In [ ]:
# Загружаем облако из датасета
cloud_idx = 0
cloud_xyzi = KITTI_DATASET.get_velo(cloud_idx)
print(type(cloud_xyzi), cloud_xyzi.shape, cloud_xyzi.dtype)

# Визуализируем облако
figure = create_plotly_figure(bgcolor='black')

colors = apply_min_max_scaling(cloud_xyzi[:, 3], min_value=0.2, max_value=1.0)
colors = colors**(2/3.)  # Гамма-коррекция цветов 
colors = convert_values_to_rgba_tuples_f64(colors, cmap='hot')
assert colors.shape == (cloud_xyzi.shape[0], 4)

labels = [f'i={intensity:3f}' for intensity in cloud_xyzi[:, 3]]

plot_cloud(
    cloud=cloud_xyzi,
    colors=colors,
    figure=figure,
    labels=labels,
).show()

del colors, labels

#### Визуализация трех последовательных облаков<sup>[toc](#_toc)</sup>

In [ ]:
cloud0 = KITTI_DATASET.get_velo(0)
cloud1 = KITTI_DATASET.get_velo(1)
cloud2 = KITTI_DATASET.get_velo(2)

figure = create_plotly_figure(bgcolor='black')
plot_cloud(cloud0, colors='red', figure=figure)
plot_cloud(cloud1, colors='green', figure=figure)
plot_cloud(cloud2, colors='blue', figure=figure)
figure.show()

del cloud0, cloud1, cloud2

<a name='kitti_localization'></a>
### Локализация<sup>[toc](#_toc)</sup>

Результаты локализации лежат в `oxts` файлах. Описание формата можно посмотреть [здесь](https://github.com/pratikac/kitti/blob/master/readme.raw.txt).

Для преобразования между различными системами геопозиционирования нам потребуется библиотека `pyproj`, установка которой проведена во время настройки питоновского окружения.

Считывание показаний из датасета состоит из несольких шагов:
1. Считываем latitude, longitude, altitude положений GPS-сенсора (LLA-геопозиции)
2. Создаем [проекцию меркатора](https://en.wikipedia.org/wiki/Mercator_projection), где в качестве референсной точки выбираем первое показание GPS-сенсора
3. Конвертируем LLA-геопозиции в XYZ-геопозиции в системе координат меркатора
4. Считываем ориентации GPS-сенсора в системе координат меркатора
5. Формируем окончательные значения показаний локализации

Эти шаги детально рассматривались на лекции/семинаре. Здесь же просто воспользуемся классом `KittiDatasetAdaptor`, чтобы просто получить окончетельный набор поз GPS-сенсора.

In [ ]:
from pylib.geo.geo_lla_xyz_converter import GeoLlaXyzConverter
from pylib.geo.geo_position_lla import GeoPositionLLA
from pylib.geo.geo_position_xyz import GeoPositionXYZ
from pylib.kitti.dataset_adaptor import KittiDatasetAdaptor
from pylib.kitti.localization import (
    Localization,
    build_localization,
    build_localizations,
)
from pylib.kitti.convert_localization import (
    convert_localization_to_transform_matrix,
    convert_localizations_to_transform_matrices,
)

In [ ]:
KITTI_DATASET_ADAPTOR = KittiDatasetAdaptor(KITTI_DATASET)
reference_point = KITTI_DATASET_ADAPTOR.read_geo_positions_lla()[0]
localizations = KITTI_DATASET_ADAPTOR.build_localizations(reference_point)
del reference_point

#### Пример работы GeoLlaXyzConverter-а

In [ ]:
# Инициализируем конвертер между LLA-геопозициями и XYZ-геопозициями в системе координат меркатора
geo_lla_xyz_converter = GeoLlaXyzConverter(geo_positions_lla[0])
print(f'geo_lla_xyz_converter.reference_point: {geo_lla_xyz_converter.reference_point}')

# Проверяем работу конвертера между проекцией меркатора и latlong-системой координат
print(geo_lla_xyz_converter.convert_xyz_to_lla(GeoPositionXYZ(0., 0., 0.)))
print(geo_lla_xyz_converter.convert_lla_to_xyz(geo_lla_xyz_converter.reference_point))
del geo_lla_xyz_converter

#### Визуализация траекторию, построенной по XYZ-позициями GPS-сенсора

In [ ]:
xs = [g.x for g in geo_positions_xyz]
ys = [g.y for g in geo_positions_xyz]
zs = [g.z for g in geo_positions_xyz]

fig = create_plotly_figure()
fig.add_scatter3d(**dict(
    x=xs,
    y=ys,
    z=zs,
    mode='lines+markers',
    line=dict(color='red', width=2),
    marker=dict(size=5),
))
fig.show()
del xs, ys, zs

<a name='kitti_calibrations'></a>
### Калибровки<sup>[toc](#_toc)</sup>
* Описание расположения сенсоров http://www.cvlibs.net/datasets/kitti/setup.php
* Описание содержимого файлов с калибровками https://github.com/yanii/kitti-pcl/blob/master/KITTI_README.TXT
    * `T_cam2_velo` &mdash; матрица перехода из лидара в камеру
    * `R_rect_20` &mdash; что это?
    * `P_rect_20` &mdash; матрица проекций ректифицированного изображения
  
 * Камера 0 выступает в качестве референсной системы координат. Положение лидара в системе координат камеры 0  содержится в файле `calib_velo_to_cam.txt` (аттрибут `T_cam0_velo_unrect`).


В KITTI в качестве ректифицированных изображений рассматриваются "выровненные" в плоскости стереопары изображения. Так как плоскость стереопары в общем случае повернута относительно исходной плоскости изображения камеры, то в дело вступает матрица ректификации. Матрица перехода из системы координат лидара в систему координат камеры $i$ имеет следующий вид:
$$
T_{l \to c_i} = R_{i} T_{c_0 \to c_i} T_{l \to c_0},
$$
где
* $T_{l \to c_0}$ &mdash; матрица перехода из системы координат лидара в систему координат камеры $0$
* $T_{c_0 \to c_i}$ &mdash; матрица перехода из системы координат лидара в систему координат камер $i$
* $R_i$ &mdash; матрица ректификации для камеры $i$ (матрица поворота, которая выравнивает камеры между собой, переводя их к одной плоскости изображения).

In [ ]:
obtained = np.dot(DATASET.calib.R_rect_00, DATASET.calib.T_cam0_velo_unrect) 
expected = DATASET.calib.T_cam0_velo
assert np.max(np.abs(expected - obtained)) < 1e-10
print(DATASET.calib.T_cam0_velo)

#### Матрицы ректификации

In [ ]:
for camera_idx in range(4):
    R_rect_name = 'R_rect_{}0'.format(camera_idx)
    print('{}:\n{}\n'.format(R_rect_name, getattr(DATASET.calib, R_rect_name)))
    del camera_idx, R_rect_name

**Матрицы ректификации у всех камер отличаются**

#### Матрицы проекций

In [ ]:
P_rect = {}
for camera_idx in range(4):
    P_rect_name = 'P_rect_{}0'.format(camera_idx)
    P_rect[camera_idx] = getattr(DATASET.calib, P_rect_name)
    print('{}:\n{}\n'.format(P_rect_name, P_rect[camera_idx]))
    del camera_idx

for first, second in itertools.product(range(4), range(4)):
    assert np.max(np.abs(P_rect[first] - P_rect[second])[:3, :3]) == 0.

* Основная часть у всех матриц проекций идентична, что, в принципе, логично, так как плоскость Z = 1 у стерео-камер идентична.
* Камера 2 единственная расположена левее камеры 0, именно поэтому элемент P[0, 3] у этой камеры положителен, в отличие от камер 1 и 3.

In [ ]:
camera_idx = 2
np.set_printoptions(precision=5)
L2C_name = f'T_cam{camera_idx}_velo'
K_name = f'K_cam{camera_idx}'
P_rect_name = f'P_rect_{camera_idx}0'
R_rect_name = f'R_rect_{camera_idx}0'

print('Lidar to camera transform:\n{}\n'.format(getattr(DATASET.calib, L2C_name)))
print('{}:\n{}\n'.format(K_name, getattr(DATASET.calib, K_name)))
print('{}:\n{}\n'.format(R_rect_name, getattr(DATASET.calib, R_rect_name)))
print('{}:\n{}\n'.format(P_rect_name, getattr(DATASET.calib, P_rect_name)))
del camera_idx

* https://medium.com/swlh/camera-lidar-projection-navigating-between-2d-and-3d-911c78167a94
* https://github.com/darylclimb/cvml_project/tree/master/projections/lidar_camera_projection

In [ ]:
from pykitti.utils import read_calib_file

CALIB = {}
CALIB['T_lidar_to_unrect0'] = DATASET.calib.T_cam0_velo_unrect

cam_to_cam_calib = read_calib_file(os.path.join(KITTI_DIR_PATH, '2011_09_26', 'calib_cam_to_cam.txt'))
for camera_index in range(4):
    T = np.eye(4)
    T[:3, :3] = cam_to_cam_calib[f'R_0{camera_index}'].reshape(3, 3)
    T[:3, 3] = cam_to_cam_calib[f'T_0{camera_index}']
    CALIB[f'T_lidar_to_unrect{camera_index}'] = np.dot(T, CALIB['T_lidar_to_unrect0'])
    
    R_rect = np.eye(4)
    R_rect[:3, :3] = cam_to_cam_calib[f'R_rect_0{camera_index}'].reshape(3, 3)
    CALIB[f'T_lidar_to_rect{camera_index}'] = np.dot(R_rect, CALIB[f'T_lidar_to_unrect{camera_index}'])

In [ ]:
NUM_CAMERAS = 4
for camera_idx in range(NUM_CAMERAS):
    print(CALIB['T_lidar_to_rect1'])
    print(DATASET.calib.T_cam1_velo)

#### Посмотрим на расположение камер

Камера 2 самая левая согласно расположению сенсоров.

In [ ]:
print(DATASET.calib.T_cam2_velo - DATASET.calib.T_cam0_velo)
print(DATASET.calib.T_cam2_velo - DATASET.calib.T_cam3_velo)
print(DATASET.calib.T_cam2_velo - DATASET.calib.T_cam1_velo)

Таким образом, все +- согласно ожидаемому расположению:
* Обе камеры стоят параллельно по осям
* Камера 2 стоит левее камеры 0 на 6.2 см
* Камера 2 стоит левее камеры 3 на 53.2 см

In [ ]:
print(DATASET.calib.P_rect_20)

In [ ]:
np.set_printoptions(precision=5)
print(DATASET.calib.T_cam2_velo)

In [ ]:
from pylib.project_points import (
    project_points_on_image,
    project_points_on_cylinder,
)

<a name='kitti_utils_projection_on_image'></a>
### Проектирование лидарных точек на изоображение<sup>[toc](#_toc)</sup>

In [ ]:
frame_idx = 34
camera_idx = 2

image = getattr(DATASET, f'get_cam{camera_idx}')(frame_idx)
cloud = DATASET.get_velo(frame_idx)
n_points = cloud.shape[0]

print('Image: type={}, size={}'.format(type(image), image.size))
print('Cloud: type={}, shape={}'.format(type(cloud), cloud.shape))

projected_points, depths, valid_points_mask = project_points_on_image(
    image=image,
    cloud=cloud,
    lidar_to_camera_transform=DATASET.calib.T_cam0_velo,
    rectification_matrix=np.eye(4),
    projection_matrix=getattr(DATASET.calib, f'P_rect_{camera_idx}0'),
)

print('Projected points shape: {}'.format(projected_points.shape))
print('Points in image canvas: {}'.format(np.sum(valid_points_mask)))

_, axes = plt.subplots(2, 1, figsize=(40, 20))

axes[0].imshow(image)
axes[1].imshow(image)
axes[1].scatter(
    projected_points[valid_points_mask, 0],
    projected_points[valid_points_mask, 1],
    c=depths[valid_points_mask],
    s=3)
del frame_idx, camera_idx

<a name='kitti_utils_projection_on_scan'></a>
### Projecting lidar points on scan<sup>[toc](#_toc)</sup>

In [ ]:
frame_idx = 34
image = DATASET.get_cam2(frame_idx)
points_xyzi = DATASET.get_velo(frame_idx)
print('Image is of type: {}'.format(type(image)))
print('Cloud is of type: {}'.format(type(points_xyzi)))

_, _, valid_points_mask = project_points_on_image(
    image=image,
    cloud=points_xyzi,
    lidar_to_camera_transform=DATASET.calib.T_cam0_velo,
    rectification_matrix=np.eye(4),
    projection_matrix=DATASET.calib.P_rect_20)
print('Points in image canvas: {}'.format(np.sum(valid_points_mask)))
print('Projected points shape: {}'.format(projected_points.shape))

valid_points_xyzi = points_xyzi[valid_points_mask]
valid_points_xyz = valid_points_xyzi[:, :3]
valid_depths = np.linalg.norm(valid_points_xyz, axis=1)
valid_intensities = valid_points_xyzi[:, 3]

num_lines = 64
num_columns = 640
scan, points_to_cylinder_map = project_points_on_cylinder(
    points=valid_points_xyzi,
    attributes=[0.05 * valid_depths, valid_intensities],
    num_lines=num_lines,
    num_columns=num_columns)
assert scan.shape == (3, num_lines, num_columns)

num_collisions = np.sum(scan[2] > 1)
num_points = np.sum(scan[2])
assert valid_points_xyz.shape[0] == num_points

print('points number:       {}'.format(points_xyzi.shape[0]))
print('valid points number: {}'.format(num_points))
print('collisions number:   {}'.format(num_collisions))
print('collisions rate:     {}'.format(float(num_collisions) / num_points))

plt.figure(figsize=(20, 10))
plt.imshow(np.sqrt(scan[0]), cmap='afmhot')

del frame_idx, image

<a name='sources'></a>
# Источники<sup>[toc](#_toc)</sup>
* [Блог Такси. Яндекс разрабатывает лидары](https://taxi.yandex.ru/blog/lidar)
* [Блог Яндекса. Беспилотный флот Яндекса перешёл на собственные лидары: почему это важно и что в них особенного](https://yandex.ru/blog/company/bespilotnyy-flot-yandeksa-pereshel-na-sobstvennye-lidary-pochemu-eto-vazhno-i-chto-v-nikh-osobennogo)

* [Хабр. Беспилотный автомобиль: оживляем алгоритмы. Доклад Яндекса](https://habr.com/ru/companies/yandex/articles/471636/)
* [Хабр. Как Яндекс делает обычные автомобили беспилотными](https://habr.com/ru/companies/yandex/articles/585444/)
* [Хабр. Нейронные сети для планирования движения беспилотных автомобилей](https://habr.com/ru/companies/yandex/articles/763348/)